# Capstone Phase 3: AI-Powered Backend Development

**Project:** TripSplit - Group Travel Expense Manager

**Objective:** Transform the Phase 2 architecture and database schema into a fully functional FastAPI backend with SQLAlchemy ORM models, Pydantic schemas, and complete CRUD endpoints.

**Estimated Time:** 120 minutes

---

## 📋 Before You Begin

**Prerequisites:**
1. ✅ Completed Phase 1 (PRD generated)
2. ✅ Completed Phase 2 (Architecture and Schema generated)
3. ✅ `.env` file in project root with your `GOOGLE_API_KEY`
4. ✅ Phase 2 artifacts exist:
   - `Artifacts/Documentation/architecture.md`
   - `artifacts/schema.sql`

---

## 📖 Introduction

This notebook implements Phase 3 of the capstone project workflow, focusing on backend development. Following the README guidance for "AI as Backend Developer", we will:

1. Load the database schema from Phase 2
2. Generate SQLAlchemy ORM models
3. Generate Pydantic validation schemas
4. Create FastAPI application boilerplate
5. Generate CRUD endpoints for all major entities
6. Set up database configuration and initialization
7. Create a working FastAPI application

All artifacts will be saved to the `backend/` folder with proper organization following FastAPI best practices.

**How to use this notebook:**
- Run **Step 0** first to install required packages
- Then run each cell in order from top to bottom
- Review generated code before saving
- Test the API endpoints after generation

## Step 0: Install Required Packages

**Run this cell FIRST** to ensure all required packages are installed in the notebook's kernel environment.

In [ ]:
import sys
import subprocess

print("Installing required packages in the current kernel environment...")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print("=" * 70)

# List of required packages for Phase 3
required_packages = [
    'python-dotenv',
    'google-genai',
    'fastapi',
    'uvicorn',
    'sqlalchemy',
    'alembic',
    'pydantic',
    'pydantic-settings',
]

# Install each package
for package in required_packages:
    print(f"\n📦 Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"   ✓ {package} installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to install {package}: {e}")

print("\n" + "=" * 70)
print("✅ Package installation complete!")
print("\nYou can now proceed to Step 1.")
print("=" * 70)

## Step 1: Setup

Initialize the environment, load API credentials, and set up the LLM client.

In [ ]:
import sys
import os
import json

# ============================================================================
# Step 1a: Find Project Root and Add to Path
# ============================================================================
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"📁 Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# ============================================================================
# Step 1b: Load Environment Variables
# ============================================================================
try:
    from dotenv import load_dotenv
    print("✓ python-dotenv is installed")
except ImportError:
    print("❌ ERROR: python-dotenv is not installed!")
    raise

env_path = os.path.join(project_root, '.env')
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"✓ Loaded .env file from: {env_path}")
else:
    print(f"⚠️  WARNING: .env file not found at: {env_path}")

google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    masked_key = f"{google_api_key[:8]}...{google_api_key[-4:]}" if len(google_api_key) > 12 else "***"
    print(f"✓ GOOGLE_API_KEY found: {masked_key}")
else:
    print("❌ ERROR: GOOGLE_API_KEY not found in environment!")

# ============================================================================
# Step 1c: Import Utils and Initialize LLM Client
# ============================================================================
try:
    from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output
    print("✓ Utils imported successfully")
except ImportError as e:
    print(f"❌ ERROR importing utils: {e}")
    raise

print("\n" + "="*60)
print("Initializing LLM Client...")
print("="*60)

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

if client is not None and api_provider is not None:
    print(f"\n✅ SUCCESS! LLM Client initialized")
    print(f"   Provider: {api_provider}")
    print(f"   Model: {model_name}")
else:
    print("\n❌ FAILED to initialize LLM client!")
    raise RuntimeError("LLM client initialization failed")

## Step 2: Load Phase 2 Artifacts

Load the database schema and architecture document that will guide backend code generation.

In [ ]:
# Load schema.sql
schema_path = "artifacts/schema.sql"
schema_content = load_artifact(schema_path)

if schema_content:
    print(f"✓ Loaded database schema from {schema_path}")
    print("\nSchema preview (first 800 characters):")
    print(schema_content[:800])
    print("\n...")
else:
    raise FileNotFoundError(f"Schema not found at {schema_path}. Run Phase 2 first.")

# Load architecture.md for context
arch_path = "Artifacts/Documentation/architecture.md"
arch_content = load_artifact(arch_path)

if arch_content:
    print(f"\n✓ Loaded architecture document from {arch_path}")
    print(f"   Document length: {len(arch_content)} characters")
else:
    print(f"⚠️  Architecture document not found at {arch_path}")
    arch_content = ""  # Continue without it

## Step 3: Generate SQLAlchemy ORM Models

Transform the SQL schema into Python SQLAlchemy ORM models with proper relationships, constraints, and type hints.

In [ ]:
sqlalchemy_models_prompt = f"""
You are an expert Python backend developer. Generate comprehensive SQLAlchemy ORM models based on the provided database schema.

## Database Schema
```sql
{schema_content}
```

## Requirements

1. Create a complete `models.py` file with all necessary imports
2. Include:
   - Proper SQLAlchemy 2.0 syntax
   - Mapped classes using `mapped_column()` and type annotations
   - All relationships with `Mapped[]` and `relationship()`
   - Back-populates for bidirectional relationships
   - Proper foreign key constraints
   - Index definitions where appropriate
   - Cascade delete behaviors
   - Table constraints (unique, check)
   - Useful `__repr__` methods
   - Timestamps (created_at, updated_at) with defaults

3. Organization:
   - Import section at top
   - Base class definition
   - Models in logical order (parent tables first)
   - Clear comments explaining complex relationships

4. Follow FastAPI/SQLAlchemy best practices:
   - Use `DeclarativeBase` for base class
   - Type hints for all columns
   - Proper enum handling if needed
   - JSON column types where appropriate

5. Models to generate (at minimum):
   - User
   - Trip
   - Participant
   - Expense
   - ExpenseSplit
   - Receipt (if applicable)
   - Payment (if applicable)
   - Any other tables from schema

Generate ONLY the Python code for `models.py`. No explanatory text before or after.
The code must be complete and ready to save.
"""

print("=" * 80)
print("GENERATING SQLALCHEMY ORM MODELS")
print("=" * 80)

sqlalchemy_models = get_completion(sqlalchemy_models_prompt, client, model_name, api_provider, temperature=0.1)
sqlalchemy_models_clean = clean_llm_output(sqlalchemy_models, language="python")

print("\n✓ SQLAlchemy models generated")
print("\nPreview (first 1000 characters):")
print(sqlalchemy_models_clean[:1000])
print("\n... (full code will be saved to file)")

## Step 4: Generate Pydantic Schemas

Create Pydantic models for request/response validation, separate from ORM models.

In [ ]:
pydantic_schemas_prompt = f"""
You are an expert Python backend developer. Generate comprehensive Pydantic schemas for API request/response validation.

## Database Schema (for reference)
```sql
{schema_content}
```

## SQLAlchemy Models (for reference)
```python
{sqlalchemy_models_clean}
```

## Requirements

1. Create a complete `schemas.py` file with Pydantic v2 models
2. For EACH entity (User, Trip, Expense, etc.), create:
   - `Base` schema (common fields)
   - `Create` schema (for POST requests, no ID)
   - `Update` schema (for PUT/PATCH, optional fields)
   - `InDB` schema (matches database, includes ID)
   - `Response` schema (for API responses, may include relationships)

3. Include:
   - Proper imports (BaseModel, Field, ConfigDict, etc.)
   - Field validators where appropriate
   - Default values
   - Optional vs required fields
   - `model_config` with `from_attributes=True` for ORM compatibility
   - Nested schemas for relationships (but avoid circular imports)
   - Appropriate constraints (min/max length, regex patterns, etc.)

4. Follow Pydantic v2 syntax:
   - Use `Field()` for metadata
   - Use `ConfigDict` instead of class Config
   - Use proper type hints

5. Example structure for User:
   ```python
   class UserBase(BaseModel):
       email: str = Field(..., description="User email")
       username: str = Field(..., min_length=3)
   
   class UserCreate(UserBase):
       password: str = Field(..., min_length=8)
   
   class UserUpdate(BaseModel):
       email: str | None = None
       username: str | None = None
   
   class UserInDB(UserBase):
       id: int
       created_at: datetime
       model_config = ConfigDict(from_attributes=True)
   
   class UserResponse(UserInDB):
       # May include relationships
       pass
   ```

Generate ONLY the Python code for `schemas.py`. No explanatory text.
The code must be complete and ready to save.
"""

print("=" * 80)
print("GENERATING PYDANTIC SCHEMAS")
print("=" * 80)

pydantic_schemas = get_completion(pydantic_schemas_prompt, client, model_name, api_provider, temperature=0.1)
pydantic_schemas_clean = clean_llm_output(pydantic_schemas, language="python")

print("\n✓ Pydantic schemas generated")
print("\nPreview (first 1000 characters):")
print(pydantic_schemas_clean[:1000])
print("\n... (full code will be saved to file)")

## Step 5: Generate Database Configuration

Create database connection setup, session management, and configuration.

In [ ]:
database_config_prompt = f"""
You are an expert Python backend developer. Generate database configuration files for a FastAPI + SQLAlchemy + SQLite application.

## Architecture Context
{arch_content[:2000] if arch_content else "Local-first prototype using SQLite with WAL mode"}

## Requirements

Generate TWO files:

### File 1: `database.py`
- SQLAlchemy engine creation with SQLite
- SessionLocal factory using sessionmaker
- Database URL from environment variable or default
- WAL mode configuration for SQLite
- Connection pooling settings
- get_db() dependency function for FastAPI
- create_tables() function to initialize database

### File 2: `config.py`
- Pydantic Settings class for configuration
- Environment variable loading
- Database URL configuration
- API settings (title, version, description)
- JWT secret key settings (if needed)
- CORS settings
- File upload settings
- AI/LLM settings (if applicable)

Example structure for database.py:
```python
from sqlalchemy import create_engine, event
from sqlalchemy.orm import sessionmaker, Session
from typing import Generator
import os

DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///./tripsplit.db")

engine = create_engine(
    DATABASE_URL,
    connect_args={{"check_same_thread": False}},
    echo=False,
)

# Enable WAL mode for SQLite
@event.listens_for(engine, "connect")
def set_sqlite_pragma(dbapi_conn, connection_record):
    cursor = dbapi_conn.cursor()
    cursor.execute("PRAGMA journal_mode=WAL")
    cursor.close()

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def get_db() -> Generator[Session, None, None]:
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def create_tables():
    from .models import Base
    Base.metadata.create_all(bind=engine)
```

Output format:
=== FILE: backend/app/database.py ===
[database.py code]

=== FILE: backend/app/config.py ===
[config.py code]

Generate complete, production-ready code for both files.
"""

print("=" * 80)
print("GENERATING DATABASE CONFIGURATION")
print("=" * 80)

db_config = get_completion(database_config_prompt, client, model_name, api_provider, temperature=0.1)

print("\n✓ Database configuration generated")
print("\nPreview (first 1000 characters):")
print(db_config[:1000])
print("\n... (full code will be saved to files)")

## Step 6: Generate CRUD Repository Functions

Create database access layer with CRUD operations for each entity.

In [ ]:
crud_repository_prompt = f"""
You are an expert Python backend developer. Generate CRUD repository functions for database operations.

## SQLAlchemy Models
```python
{sqlalchemy_models_clean[:2000]}
```

## Requirements

Create a `crud.py` file with repository functions for each major entity.

For EACH entity (User, Trip, Expense, Participant, etc.), implement:
1. `create_[entity]` - Create new record
2. `get_[entity]` - Get single record by ID
3. `get_[entity]s` - Get multiple records with pagination
4. `update_[entity]` - Update existing record
5. `delete_[entity]` - Delete record
6. Additional specialized queries as needed

Example structure:
```python
from sqlalchemy.orm import Session
from . import models, schemas
from typing import List, Optional

# User CRUD
def create_user(db: Session, user: schemas.UserCreate) -> models.User:
    db_user = models.User(**user.model_dump())
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

def get_user(db: Session, user_id: int) -> Optional[models.User]:
    return db.query(models.User).filter(models.User.id == user_id).first()

def get_users(db: Session, skip: int = 0, limit: int = 100) -> List[models.User]:
    return db.query(models.User).offset(skip).limit(limit).all()

def update_user(db: Session, user_id: int, user: schemas.UserUpdate) -> Optional[models.User]:
    db_user = get_user(db, user_id)
    if db_user:
        update_data = user.model_dump(exclude_unset=True)
        for key, value in update_data.items():
            setattr(db_user, key, value)
        db.commit()
        db.refresh(db_user)
    return db_user

def delete_user(db: Session, user_id: int) -> bool:
    db_user = get_user(db, user_id)
    if db_user:
        db.delete(db_user)
        db.commit()
        return True
    return False
```

Additional functions to include:
- `get_user_by_email(db, email)` - Find user by email
- `get_trip_participants(db, trip_id)` - Get all participants for a trip
- `get_trip_expenses(db, trip_id)` - Get all expenses for a trip
- `get_user_trips(db, user_id)` - Get all trips for a user
- `calculate_trip_balances(db, trip_id)` - Calculate who owes whom
- Any other specialized queries from the PRD

Generate ONLY the Python code for `crud.py`. Make it comprehensive.
"""

print("=" * 80)
print("GENERATING CRUD REPOSITORY FUNCTIONS")
print("=" * 80)

crud_code = get_completion(crud_repository_prompt, client, model_name, api_provider, temperature=0.1)
crud_code_clean = clean_llm_output(crud_code, language="python")

print("\n✓ CRUD repository functions generated")
print("\nPreview (first 1000 characters):")
print(crud_code_clean[:1000])
print("\n... (full code will be saved to file)")

## Step 7: Generate FastAPI Router Endpoints

Create FastAPI routers with REST API endpoints for all entities.

In [ ]:
api_routers_prompt = f"""
You are an expert Python backend developer. Generate FastAPI routers with complete REST API endpoints.

## Pydantic Schemas
```python
{pydantic_schemas_clean[:2000]}
```

## CRUD Functions
```python
{crud_code_clean[:2000]}
```

## Requirements

Generate separate router files for each major domain:
1. `routers/users.py` - User management endpoints
2. `routers/trips.py` - Trip CRUD endpoints
3. `routers/expenses.py` - Expense management
4. `routers/participants.py` - Participant management
5. `routers/settlements.py` - Balance calculations and settlements

Each router should include:
- Proper HTTP methods (GET, POST, PUT, DELETE)
- Path parameters and query parameters
- Request/response schema validation
- Database dependency injection
- Error handling (404, 400, etc.)
- Status codes (200, 201, 204, 404, 422)
- API documentation (docstrings, response_model)
- Pagination support where appropriate

Example structure for users.py:
```python
from fastapi import APIRouter, Depends, HTTPException, status
from sqlalchemy.orm import Session
from typing import List
from .. import crud, schemas
from ..database import get_db

router = APIRouter(prefix="/users", tags=["users"])

@router.post("/", response_model=schemas.UserResponse, status_code=status.HTTP_201_CREATED)
def create_user(user: schemas.UserCreate, db: Session = Depends(get_db)):
    """Create a new user."""
    db_user = crud.get_user_by_email(db, email=user.email)
    if db_user:
        raise HTTPException(status_code=400, detail="Email already registered")
    return crud.create_user(db=db, user=user)

@router.get("/{user_id}", response_model=schemas.UserResponse)
def read_user(user_id: int, db: Session = Depends(get_db)):
    """Get a user by ID."""
    db_user = crud.get_user(db, user_id=user_id)
    if db_user is None:
        raise HTTPException(status_code=404, detail="User not found")
    return db_user

@router.get("/", response_model=List[schemas.UserResponse])
def read_users(skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    """Get all users with pagination."""
    users = crud.get_users(db, skip=skip, limit=limit)
    return users

@router.put("/{user_id}", response_model=schemas.UserResponse)
def update_user(user_id: int, user: schemas.UserUpdate, db: Session = Depends(get_db)):
    """Update a user."""
    db_user = crud.update_user(db, user_id=user_id, user=user)
    if db_user is None:
        raise HTTPException(status_code=404, detail="User not found")
    return db_user

@router.delete("/{user_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_user(user_id: int, db: Session = Depends(get_db)):
    """Delete a user."""
    success = crud.delete_user(db, user_id=user_id)
    if not success:
        raise HTTPException(status_code=404, detail="User not found")
    return None
```

Output format:
=== FILE: backend/app/routers/users.py ===
[users.py code]

=== FILE: backend/app/routers/trips.py ===
[trips.py code]

=== FILE: backend/app/routers/expenses.py ===
[expenses.py code]

=== FILE: backend/app/routers/participants.py ===
[participants.py code]

=== FILE: backend/app/routers/settlements.py ===
[settlements.py code]

Generate complete, production-ready code for ALL router files.
"""

print("=" * 80)
print("GENERATING FASTAPI ROUTERS")
print("=" * 80)

api_routers = get_completion(api_routers_prompt, client, model_name, api_provider, temperature=0.1)

print("\n✓ FastAPI routers generated")
print("\nPreview (first 1000 characters):")
print(api_routers[:1000])
print("\n... (full code will be saved to files)")

## Step 8: Generate Main FastAPI Application

Create the main application file that ties everything together.

In [ ]:
main_app_prompt = f"""
You are an expert Python backend developer. Generate the main FastAPI application file.

## Architecture Context
{arch_content[:2000] if arch_content else "Local-first prototype for TripSplit"}

## Requirements

Create `main.py` with:
1. FastAPI app initialization with metadata
2. CORS middleware configuration
3. Router inclusions for all domains
4. Startup/shutdown event handlers
5. Database initialization
6. Health check endpoint
7. Root endpoint with API info
8. Exception handlers
9. Logging configuration

Example structure:
```python
from fastapi import FastAPI, Request, status
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import logging

from .database import create_tables
from .config import settings
from .routers import users, trips, expenses, participants, settlements

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create FastAPI app
app = FastAPI(
    title="TripSplit API",
    description="Group Travel Expense Manager",
    version="1.0.0",
    docs_url="/docs",
    redoc_url="/redoc",
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "http://localhost:5173"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Include routers
app.include_router(users.router)
app.include_router(trips.router)
app.include_router(expenses.router)
app.include_router(participants.router)
app.include_router(settlements.router)

@app.on_event("startup")
def on_startup():
    logger.info("Starting TripSplit API...")
    create_tables()
    logger.info("Database tables created")

@app.on_event("shutdown")
def on_shutdown():
    logger.info("Shutting down TripSplit API...")

@app.get("/")
def read_root():
    return {{
        "message": "Welcome to TripSplit API",
        "version": "1.0.0",
        "docs": "/docs",
    }}

@app.get("/health")
def health_check():
    return {{"status": "healthy"}}

@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    logger.error(f"Global exception: {{exc}}")
    return JSONResponse(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        content={{"detail": "Internal server error"}},
    )
```

Also generate a `__init__.py` file for the app package.

Output format:
=== FILE: backend/app/main.py ===
[main.py code]

=== FILE: backend/app/__init__.py ===
[__init__.py code]

Generate complete, production-ready code.
"""

print("=" * 80)
print("GENERATING MAIN APPLICATION")
print("=" * 80)

main_app = get_completion(main_app_prompt, client, model_name, api_provider, temperature=0.1)

print("\n✓ Main application generated")
print("\nPreview (first 1000 characters):")
print(main_app[:1000])
print("\n... (full code will be saved to files)")

## Step 9: Save All Generated Code

Save all generated code files to the proper backend directory structure.

In [ ]:
print("=" * 80)
print("SAVING ALL GENERATED CODE")
print("=" * 80)

# Save SQLAlchemy models
save_artifact(sqlalchemy_models_clean, "backend/app/models.py")
print("✓ Saved backend/app/models.py")

# Save Pydantic schemas
save_artifact(pydantic_schemas_clean, "backend/app/schemas.py")
print("✓ Saved backend/app/schemas.py")

# Save database configuration
# Parse the multi-file output
if "=== FILE:" in db_config:
    files = db_config.split("=== FILE:")
    for file_section in files[1:]:  # Skip first empty split
        lines = file_section.strip().split("\n", 1)
        if len(lines) == 2:
            file_path = lines[0].strip().replace("===", "").strip()
            file_content = lines[1].strip()
            file_content = clean_llm_output(file_content, language="python")
            save_artifact(file_content, file_path)
            print(f"✓ Saved {file_path}")
else:
    # Single file, assume database.py
    db_config_clean = clean_llm_output(db_config, language="python")
    save_artifact(db_config_clean, "backend/app/database.py")
    print("✓ Saved backend/app/database.py")

# Save CRUD functions
save_artifact(crud_code_clean, "backend/app/crud.py")
print("✓ Saved backend/app/crud.py")

# Save routers
if "=== FILE:" in api_routers:
    files = api_routers.split("=== FILE:")
    for file_section in files[1:]:
        lines = file_section.strip().split("\n", 1)
        if len(lines) == 2:
            file_path = lines[0].strip().replace("===", "").strip()
            file_content = lines[1].strip()
            file_content = clean_llm_output(file_content, language="python")
            save_artifact(file_content, file_path)
            print(f"✓ Saved {file_path}")

# Save main application
if "=== FILE:" in main_app:
    files = main_app.split("=== FILE:")
    for file_section in files[1:]:
        lines = file_section.strip().split("\n", 1)
        if len(lines) == 2:
            file_path = lines[0].strip().replace("===", "").strip()
            file_content = lines[1].strip()
            file_content = clean_llm_output(file_content, language="python")
            save_artifact(file_content, file_path)
            print(f"✓ Saved {file_path}")

# Create __init__.py files for packages
save_artifact("", "backend/__init__.py")
print("✓ Saved backend/__init__.py")

save_artifact("", "backend/app/__init__.py")
print("✓ Saved backend/app/__init__.py")

save_artifact("", "backend/app/routers/__init__.py")
print("✓ Saved backend/app/routers/__init__.py")

print("\n" + "=" * 80)
print("✅ ALL CODE SAVED SUCCESSFULLY")
print("=" * 80)

## Step 10: Generate Requirements and Documentation

Create requirements.txt and basic API documentation.

In [ ]:
requirements_content = """# FastAPI Backend Requirements
fastapi==0.104.1
uvicorn[standard]==0.24.0
sqlalchemy==2.0.23
alembic==1.12.1
pydantic==2.5.0
pydantic-settings==2.1.0
python-dotenv==1.0.0
python-multipart==0.0.6

# Database
aiosqlite==0.19.0

# Authentication (if needed)
python-jose[cryptography]==3.3.0
passlib[bcrypt]==1.7.4

# Testing
pytest==7.4.3
pytest-asyncio==0.21.1
httpx==0.25.2
"""

save_artifact(requirements_content, "backend/requirements.txt")
print("✓ Saved backend/requirements.txt")

readme_content = """# TripSplit Backend API

FastAPI backend for the TripSplit group travel expense manager.

## Setup

1. Create virtual environment:
```bash
cd backend
python -m venv venv
source venv/bin/activate  # On Windows: venv\\Scripts\\activate
```

2. Install dependencies:
```bash
pip install -r requirements.txt
```

3. Create `.env` file in project root:
```
DATABASE_URL=sqlite:///./tripsplit.db
```

4. Run the application:
```bash
uvicorn app.main:app --reload --port 8000
```

5. Access the API:
- API: http://localhost:8000
- Interactive docs: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

## Project Structure

```
backend/
├── app/
│   ├── __init__.py
│   ├── main.py           # FastAPI app
│   ├── config.py         # Configuration
│   ├── database.py       # Database setup
│   ├── models.py         # SQLAlchemy models
│   ├── schemas.py        # Pydantic schemas
│   ├── crud.py           # Database operations
│   └── routers/          # API endpoints
│       ├── __init__.py
│       ├── users.py
│       ├── trips.py
│       ├── expenses.py
│       ├── participants.py
│       └── settlements.py
├── requirements.txt
└── README.md
```

## API Endpoints

### Users
- `POST /users/` - Create user
- `GET /users/` - List users
- `GET /users/{id}` - Get user
- `PUT /users/{id}` - Update user
- `DELETE /users/{id}` - Delete user

### Trips
- `POST /trips/` - Create trip
- `GET /trips/` - List trips
- `GET /trips/{id}` - Get trip
- `PUT /trips/{id}` - Update trip
- `DELETE /trips/{id}` - Delete trip

### Expenses
- `POST /expenses/` - Create expense
- `GET /expenses/` - List expenses
- `GET /expenses/{id}` - Get expense
- `PUT /expenses/{id}` - Update expense
- `DELETE /expenses/{id}` - Delete expense

### Settlements
- `GET /settlements/trip/{trip_id}/balances` - Get trip balances
- `GET /settlements/trip/{trip_id}/optimal` - Get optimal settlement plan

See the interactive API documentation at `/docs` for complete details.
"""

save_artifact(readme_content, "backend/README.md")
print("✓ Saved backend/README.md")

print("\n✅ Requirements and documentation generated")

## Summary: Phase 3 Complete! 🎉

We have successfully completed Phase 3 of the capstone project for **TripSplit**. Here's what we've generated:

### Generated Artifacts:

1. **SQLAlchemy ORM Models**: `backend/app/models.py`
   - Complete database models with relationships
   - Proper foreign keys and constraints
   - Type hints and documentation

2. **Pydantic Schemas**: `backend/app/schemas.py`
   - Request/response validation models
   - Create, Update, InDB, Response variants for each entity
   - Field validators and constraints

3. **Database Configuration**: `backend/app/database.py` & `backend/app/config.py`
   - SQLite connection with WAL mode
   - Session management
   - Environment-based configuration

4. **CRUD Operations**: `backend/app/crud.py`
   - Repository functions for all entities
   - Specialized query functions
   - Pagination support

5. **API Routers**:
   - `backend/app/routers/users.py` - User management
   - `backend/app/routers/trips.py` - Trip CRUD
   - `backend/app/routers/expenses.py` - Expense tracking
   - `backend/app/routers/participants.py` - Participant management
   - `backend/app/routers/settlements.py` - Balance calculations

6. **Main Application**: `backend/app/main.py`
   - FastAPI app initialization
   - Router integration
   - CORS and middleware
   - Health check endpoints

7. **Supporting Files**:
   - `backend/requirements.txt` - Python dependencies
   - `backend/README.md` - Setup and usage instructions
   - Package `__init__.py` files

### Backend Directory Structure:
```
backend/
├── app/
│   ├── __init__.py
│   ├── main.py
│   ├── config.py
│   ├── database.py
│   ├── models.py
│   ├── schemas.py
│   ├── crud.py
│   └── routers/
│       ├── __init__.py
│       ├── users.py
│       ├── trips.py
│       ├── expenses.py
│       ├── participants.py
│       └── settlements.py
├── requirements.txt
└── README.md
```

### Next Steps:

1. **Test the Backend**:
   ```bash
   cd backend
   python -m venv venv
   source venv/bin/activate
   pip install -r requirements.txt
   uvicorn app.main:app --reload --port 8000
   ```
   Then visit http://localhost:8000/docs

2. **Phase 4**: Generate tests and security review
   - Unit tests with pytest
   - Integration tests
   - Security vulnerability scan

3. **Phase 5**: Build React frontend
   - Component generation from mockups
   - API integration
   - State management

### Key Features Implemented:
- ✅ Complete REST API with CRUD operations
- ✅ SQLite database with SQLAlchemy ORM
- ✅ Request/response validation with Pydantic
- ✅ Automatic API documentation
- ✅ Database relationships and constraints
- ✅ Error handling and status codes
- ✅ CORS support for frontend integration
- ✅ Health check endpoints
- ✅ Pagination support

### Testing Your API:

Once the server is running, you can test endpoints:

```bash
# Create a user
curl -X POST http://localhost:8000/users/ \
  -H "Content-Type: application/json" \
  -d '{"email": "user@example.com", "username": "testuser", "password": "password123"}'

# Get all users
curl http://localhost:8000/users/

# Health check
curl http://localhost:8000/health
```

Or use the interactive documentation at http://localhost:8000/docs to test all endpoints.

**Congratulations!** You now have a fully functional FastAPI backend ready for integration with a frontend and further testing.